<a href="https://colab.research.google.com/github/isabellacadisco/Algorithms-for-Massive-Data-course/blob/main/LSH8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
#set up os env
os.environ['KAGGLE_USERNAME'] = "isabellacadisco"
os.environ['KAGGLE_KEY'] = "847325691c2399e41dc9c0b78fbe499e"

# download dataset from kaggle and save it in drive
!kaggle datasets download -d xhlulu/medal-emnlp

100% 6.81G/6.82G [01:18<00:00, 65.6MB/s]
100% 6.82G/6.82G [01:19<00:00, 92.7MB/s]


In [3]:
!unzip medal-emnlp.zip #unzip dataset

Archive:  medal-emnlp.zip
  inflating: full_data.csv           
  inflating: pretrain_subset/test.csv  
  inflating: pretrain_subset/train.csv  
  inflating: pretrain_subset/valid.csv  


In [4]:
# set up spark
!apt-get install openjdk-8-jdk-headless -qq > /dev/null #install java as VM
!wget -q https://dlcdn.apache.org/spark/spark-3.4.0/spark-3.4.0-bin-hadoop3.tgz #download Spark
!tar xf spark-3.4.0-bin-hadoop3.tgz #unzip the file
!pip install -q findspark

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.0-bin-hadoop3"

import findspark
findspark.init("spark-3.4.0-bin-hadoop3")# SPARK_HOME
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate() #SPARK SESSION

import pyspark
type(spark)

pyspark.sql.session.SparkSession

In [5]:
file_path = './full_data.csv'

In [6]:
data_df = spark.read.csv(file_path, header=True)

In [7]:
from pyspark.sql.functions import monotonically_increasing_id

In [8]:
dataset = data_df.select('TEXT').withColumn('doc_id', monotonically_increasing_id())

In [9]:
small_dataset = dataset.sample(withReplacement=False, fraction=0.000007, seed=42)

# Text preprocessing

In [10]:
# TEXT PROCESSOR
!pip install spacy
!python -m spacy download en_core_web_sm

# Import spaCy and download the English model
import spacy
import en_core_web_sm

# Load the spaCy English model FORSE NON SERVE BOH
nlp = en_core_web_sm.load()

import string


2023-09-14 15:03:21.527161: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-09-14 15:03:23.259783: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 31.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [11]:
def preprocess_text(text):
    # Initialize spaCy resources
    nlp = spacy.load("en_core_web_sm")

    # Apply spaCy NLP pipeline
    doc = nlp(text)

    # Lemmatize and remove stopwords
    tokens = [token.lemma_.lower() for token in doc if not token.is_stop and token.text not in string.punctuation]

    # Join tokens back into a text
    cleaned_text = " ".join(tokens)

    return cleaned_text

In [12]:
# UDF --

from pyspark.sql.functions import udf
from pyspark.sql.types import StringType

# Register the UDF with Spark
preprocess_udf_spark = udf(preprocess_text, StringType())

In [13]:
df_prep = small_dataset.withColumn("preprocessed_text", preprocess_udf_spark(small_dataset['TEXT']))

In [14]:
df = df_prep.select('doc_id','preprocessed_text')

In [22]:
df.show()

+------------+--------------------+
|      doc_id|   preprocessed_text|
+------------+--------------------+
|      106539|recent outbreak s...|
| 25769866227|wefast gradientec...|
| 42949764250|paper deal patien...|
| 51539638497|close association...|
| 51539642085|occasional unpred...|
| 77309421895|relative percent ...|
|103079248410|mix alloy dental ...|
|103079293994|high incidence go...|
|103079331152|housetohouse surv...|
|120259124985|characterize cont...|
|120259129263|study efficacy ca...|
|137439020479|effect clonidine ...|
|154618911990|lipopolysaccharid...|
|154618921437|case report yearo...|
|163208873442|study af influenc...|
|188978671675|rat fetuse day ge...|
|206158468968|catenin polypepti...|
|214748392950|tissue process el...|
|214748459668|ss concentration ...|
|223338408901|sensitivity pcd w...|
+------------+--------------------+
only showing top 20 rows



### avg len docs = 673 characters

k = 5

- hash to 2^32 buckets (# poss shingles > #buckets)

- hash to 2^16 (<)

# Hash Family

In [15]:
import re, hashlib, math, time
from random import randint, seed

# scoprire esattamente a cosa serve
seed(16)

In [115]:
class hashFamily:
    def __init__(self, i):
        self.resultSize = 4 # how many bytes we want back
        self.maxLen = 20 # how long can our i be (in decimal)
        self.salt = str(i).zfill(self.maxLen)[-self.maxLen:]
        self.id = i

    def get_hash_value(self, el_to_hash):
        return int(hashlib.sha1(str(el_to_hash).encode('utf-8') + self.salt.encode('utf-8')).hexdigest()[-self.resultSize:], 16)


In [ ]:
my_hash_fun = hashFamily()

# Shingler

create k-grams and hash them

default k = 5, else specify

In [116]:
# SHINGLER


class shingler:
    def __init__(self, k = 5):

        if k > 0:
            self.k = int(k)
        else:
            self.k = 5

    #inner class utility
    #def process_doc(self, document):
     #   return re.sub("( )+|(\n)+"," ",document).lower()

    def get_shingles(self, document):
        shingles = set()
        for i in range(0, len(document)-self.k+1 ):
            shingles.add(document[i:i+self.k])
        return shingles

    def get_k(self):
        return self.k

    #return sorted hash
    def get_hashed_shingles(self, shingles_set):
        hash_function = hashFamily(0)
        return sorted( {hash_function.get_hash_value(s) for s in shingles_set} )

In [142]:
sh = shingler()

In [143]:
sh.get_k()

5

In [144]:
sh.get_shingles('ciao come va male')

{' come',
 ' male',
 ' va m',
 'a mal',
 'ao co',
 'ciao ',
 'come ',
 'e va ',
 'iao c',
 'me va',
 'o com',
 'ome v',
 'va ma'}

In [145]:
sh.get_hashed_shingles(sh.get_shingles('ciao come va male'))

[4674,
 21542,
 22610,
 40768,
 42718,
 44514,
 46426,
 49077,
 50732,
 53136,
 57526,
 59146,
 61046]

In [148]:
from pyspark import SparkContext

In [149]:
sc = SparkContext()

ValueError: ignored

In [151]:
from pyspark.sql import Row

In [189]:
dat5 = [
    Row(doc_id=1, preprocessed_text="ciao come va male"),
    Row(doc_id=2, preprocessed_text="il patio è bello"),
    Row(doc_id=3, preprocessed_text="oggi è stata una giornata di merda"),
    Row(doc_id=4, preprocessed_text="il patio è bello")

]

In [190]:
test_df = spark.createDataFrame(dat5)

In [191]:
test_df.show()

+------+--------------------+
|doc_id|   preprocessed_text|
+------+--------------------+
|     1|   ciao come va male|
|     2|    il patio è bello|
|     3|oggi è stata una ...|
|     4|    il patio è bello|
+------+--------------------+



In [192]:
def shingling_map(row):
    out = []
    sh_instance = shingler()
    hashed_shingles = sh_instance.get_hashed_shingles( sh_instance.get_shingles( row['preprocessed_text'] ) )
    signature_size = 300
    for i in range(0,signature_size): #signature size
        out.append( (row['doc_id'], hashed_shingles, i ) )
    #return an iterator to use flatMap => produce more than one key-value pair as output (namely one per hash function)
    return iter(out)

In [193]:
prova = test_df.rdd.flatMap(shingling_map)

In [194]:
prova.toDF().show()

+---+--------------------+---+
| _1|                  _2| _3|
+---+--------------------+---+
|  1|[4674, 21542, 226...|  0|
|  1|[4674, 21542, 226...|  1|
|  1|[4674, 21542, 226...|  2|
|  1|[4674, 21542, 226...|  3|
|  1|[4674, 21542, 226...|  4|
|  1|[4674, 21542, 226...|  5|
|  1|[4674, 21542, 226...|  6|
|  1|[4674, 21542, 226...|  7|
|  1|[4674, 21542, 226...|  8|
|  1|[4674, 21542, 226...|  9|
|  1|[4674, 21542, 226...| 10|
|  1|[4674, 21542, 226...| 11|
|  1|[4674, 21542, 226...| 12|
|  1|[4674, 21542, 226...| 13|
|  1|[4674, 21542, 226...| 14|
|  1|[4674, 21542, 226...| 15|
|  1|[4674, 21542, 226...| 16|
|  1|[4674, 21542, 226...| 17|
|  1|[4674, 21542, 226...| 18|
|  1|[4674, 21542, 226...| 19|
+---+--------------------+---+
only showing top 20 rows



In [195]:
def minhash_map(docId_ShingleSet_hFunct):
    doc_id = docId_ShingleSet_hFunct[0]
    shingles = docId_ShingleSet_hFunct[1]
    hash_f = hashFamily( docId_ShingleSet_hFunct[2] )
    min_h = math.inf
    for el in shingles:
        hash_value = hash_f.get_hash_value(el)
        if hash_value < min_h:
            min_h = hash_value

    return (doc_id, min_h )

In [196]:
sig_matrix_prova = prova.map(minhash_map).groupByKey().map(lambda x : (x[0], list(x[1])))

In [197]:
sig_matrix_prova.toDF().show()

+---+--------------------+
| _1|                  _2|
+---+--------------------+
|  2|[6993, 56, 7046, ...|
|  4|[6993, 56, 7046, ...|
|  1|[2170, 887, 340, ...|
|  3|[24, 1968, 2428, ...|
+---+--------------------+



In [198]:
def map_buckets(row):

    band_nr = 30
    row_nr = 10
    doc_id = row[0]
    doc_sign = row[1]
    hash_funct = hashFamily(1)
    out = []

    for i in range(0,band_nr):
        band_id = i
        idx = i*row_nr
        set_col = ' '.join(str(x) for x in doc_sign[idx:idx+row_nr])
        bucket = hash_funct.get_hash_value(set_col)
        out.append( ( (band_id, bucket), doc_id)  )

    return iter(out) #since we are going to return multiple tuple we call flatMap() and return an iterator on those tuple


In [199]:
candidates_prova = sig_matrix_prova.flatMap(map_buckets).groupByKey().map(lambda x : (x[0], list(x[1])))

In [200]:
candidates_prova.toDF().show()

+-----------+------+
|         _1|    _2|
+-----------+------+
| {3, 34295}|[2, 4]|
| {7, 23517}|[2, 4]|
| {9, 55389}|[2, 4]|
|{14, 52888}|[2, 4]|
| {15, 5903}|[2, 4]|
|{16, 57426}|[2, 4]|
|{18, 39782}|[2, 4]|
|{19, 48173}|[2, 4]|
| {24, 1832}|[2, 4]|
|{26, 17238}|[2, 4]|
|  {1, 3139}|   [1]|
| {2, 58348}|   [1]|
| {3, 52929}|   [1]|
| {5, 56743}|   [1]|
|{11, 43801}|   [1]|
|{12, 25264}|   [1]|
|{13, 28351}|   [1]|
|{18, 65006}|   [1]|
| {19, 4707}|   [1]|
|{20, 37940}|   [1]|
+-----------+------+
only showing top 20 rows



In [201]:
def generate_pairs(rdd):
    # Function to generate pairs from a list while considering (a, b) the same as (b, a)
    def generate_unique_pairs(values):
        pairs = list(combinations(values, 2))
        #sorted_pairs = [tuple(sorted(pair)) for pair in pairs]
        return pairs

    # Generate pairs for each row
    pairs_rdd = rdd.flatMap(lambda row: generate_unique_pairs(row[1]))

    # Collect all pairs and deduplicate them
    unique_pairs = pairs_rdd.collect()
    unique_pairs = list(set(unique_pairs))

    return unique_pairs

In [202]:
unique_pairs_prova = generate_pairs(candidates_prova)

In [203]:
print(unique_pairs_prova)

[(2, 4)]


In [118]:
hashed_val  = df.rdd.flatMap(shingling_map)

In [68]:
rdd3.toDF().show()

+------+--------------------+---+
|    _1|                  _2| _3|
+------+--------------------+---+
|106539|[3, 5, 6, 7, 8, 9...|  0|
|106539|[3, 5, 6, 7, 8, 9...|  1|
|106539|[3, 5, 6, 7, 8, 9...|  2|
|106539|[3, 5, 6, 7, 8, 9...|  3|
|106539|[3, 5, 6, 7, 8, 9...|  4|
|106539|[3, 5, 6, 7, 8, 9...|  5|
|106539|[3, 5, 6, 7, 8, 9...|  6|
|106539|[3, 5, 6, 7, 8, 9...|  7|
|106539|[3, 5, 6, 7, 8, 9...|  8|
|106539|[3, 5, 6, 7, 8, 9...|  9|
|106539|[3, 5, 6, 7, 8, 9...| 10|
|106539|[3, 5, 6, 7, 8, 9...| 11|
|106539|[3, 5, 6, 7, 8, 9...| 12|
|106539|[3, 5, 6, 7, 8, 9...| 13|
|106539|[3, 5, 6, 7, 8, 9...| 14|
|106539|[3, 5, 6, 7, 8, 9...| 15|
|106539|[3, 5, 6, 7, 8, 9...| 16|
|106539|[3, 5, 6, 7, 8, 9...| 17|
|106539|[3, 5, 6, 7, 8, 9...| 18|
|106539|[3, 5, 6, 7, 8, 9...| 19|
+------+--------------------+---+
only showing top 20 rows



# Min Hash

In [119]:
def minhash_map(docId_ShingleSet_hFunct):
    doc_id = docId_ShingleSet_hFunct[0]
    shingles = docId_ShingleSet_hFunct[1]
    hash_f = hashFamily( docId_ShingleSet_hFunct[2] )
    min_h = math.inf
    for el in shingles:
        hash_value = hash_f.get_hash_value(el)
        if hash_value < min_h:
            min_h = hash_value

    return (doc_id, min_h )

In [120]:
s_m = hashed_val.map(minhash_map).groupByKey().map(lambda x : (x[0], list(x[1])))

In [121]:
s_m.toDF().show()

+------------+--------------------+
|          _1|                  _2|
+------------+--------------------+
|558345788776|[15, 39, 687, 68,...|
|575525688293|[154, 121, 223, 4...|
|240518254693|[185, 73, 23, 407...|
|532576069849|[2247, 2088, 563,...|
|730144492409|[69, 403, 223, 16...|
|962072799544|[15, 109, 874, 88...|
| 77309421895|[275, 179, 122, 9...|
|292057778213|[520, 180, 548, 1...|
|876173412253|[13, 57, 19, 474,...|
|712964626235|[21, 97, 19, 25, ...|
|120259124985|[41, 52, 17, 47, ...|
|103079248410|[63, 121, 262, 36...|
|498216207321|[748, 77, 412, 19...|
|231928251784|[49, 29, 19, 36, ...|
|481036462278|[131, 52, 26, 147...|
|618475337381|[491, 121, 280, 8...|
|712964604209|[40, 80, 1145, 65...|
|214748392950|[27, 74, 67, 18, ...|
|317827674567|[47, 56, 9, 191, ...|
|472446497551|[60, 77, 597, 214...|
+------------+--------------------+
only showing top 20 rows



In [54]:
sig_matrix3.count()

94

# Banding technique
choice of b and r

In [63]:
from scipy.optimize import fsolve

# Define a function that represents the system of equations
def equations(vars):
    b, r = vars
    eq1 = 0.7 - ((1 / b) ** (1 / r))
    eq2 = b * r - 300
    return [eq1, eq2]

# Initial guess for the solution
initial_guess = [1.0, 1.0]  # You can change these initial values

# Solve the system of equations
result = fsolve(equations, initial_guess)

# Extract the values of b and r from the result
b_solution, r_solution = result

# Print the solutions
print(f"Approximate b: {b_solution}")
print(f"Approximate r: {r_solution}")


Approximate b: 31.123733607951603
Approximate r: 9.638946399547725


In [64]:
b = 30
r = 10

t = ((1 / b) ** (1 / r))

print(t)

0.7116851017915987


# LSH

In [71]:
def map_buckets(row):

    band_nr = 30
    row_nr = 10
    doc_id = row[0]
    doc_sign = row[1]
    hash_funct = hashFamily(1)
    out = []

    for i in range(0,band_nr):
        band_id = i
        idx = i*row_nr
        set_col = ' '.join(str(x) for x in doc_sign[idx:idx+row_nr])
        bucket = hash_funct.get_hash_value(set_col)
        out.append( ( (band_id, bucket), doc_id)  )

    return iter(out) #since we are going to return multiple tuple we call flatMap() and return an iterator on those tuple


In [73]:
candidates = sig_matrix3.flatMap(map_buckets).groupByKey().map(lambda x : (x[0], list(x[1])))

In [130]:
candidates_4bytes = s_m.flatMap(map_buckets).groupByKey().map(lambda x : (x[0], list(x[1])))

In [93]:
candidates.first()[1]

[575525688293, 541165929865, 188978671675]

In [82]:
candidates.toDF().show()

+---------+--------------------+
|       _1|                  _2|
+---------+--------------------+
|  {23, 4}|[575525688293, 54...|
|{13, 253}|[77309421895, 738...|
|  {4, 89}|[618475337381, 76...|
|{20, 209}|[51539642085, 206...|
|{10, 171}|[103079331152, 18...|
| {22, 18}|      [644245161664]|
|  {5, 87}|[773094170356, 96...|
|{17, 116}|      [824633837834]|
| {10, 56}|      [687194784093]|
|  {8, 90}|      [515396099449]|
|{12, 142}|            [106539]|
| {7, 110}|      [472446497290]|
|{13, 241}|      [730144492409]|
| {24, 94}|[712964604209, 10...|
| {7, 237}|[472446497551, 42...|
|{22, 248}|      [790274097403]|
| {6, 134}|[42949764250, 953...|
| {26, 87}|       [42949764250]|
|  {7, 35}|[283467962563, 91...|
|{28, 249}|      [188978671675]|
+---------+--------------------+
only showing top 20 rows



In [132]:
candidates_4bytes.toDF().show()

+-----------+--------------+
|         _1|            _2|
+-----------+--------------+
| {4, 42363}|[575525688293]|
| {0, 40052}|[532576069849]|
| {2, 23298}|[120259124985]|
|{19, 63222}|[231928251784]|
|{10, 12239}|[214748392950]|
|{25, 31758}|[472446497551]|
|{14, 48261}|[790274097403]|
|{21, 26601}|[154618921437]|
| {6, 10939}|[816043789573]|
|{25, 15971}|[188978671675]|
|{16, 32761}|[635655219536]|
|{28, 19693}|[635655219536]|
|{16, 26936}|[395136997960]|
|{10, 38689}|[386547161599]|
|{11, 32571}|[730144517105]|
|{14, 48841}|[773094170356]|
| {14, 9575}|[412316978617]|
| {8, 62011}|[163208873442]|
| {5, 16538}|[154618911990]|
| {9, 41379}|[721554514641]|
+-----------+--------------+
only showing top 20 rows



In [83]:
from itertools import combinations

In [94]:
def generate_pairs(rdd):
    # Function to generate pairs from a list while considering (a, b) the same as (b, a)
    def generate_unique_pairs(values):
        pairs = list(combinations(values, 2))
        #sorted_pairs = [tuple(sorted(pair)) for pair in pairs]
        return pairs

    # Generate pairs for each row
    pairs_rdd = rdd.flatMap(lambda row: generate_unique_pairs(row[1]))

    # Collect all pairs and deduplicate them
    unique_pairs = pairs_rdd.collect()
    unique_pairs = list(set(unique_pairs))

    return unique_pairs

In [95]:
unique_pairs = generate_pairs(candidates)

In [133]:
unique_pairs_4 = generate_pairs(candidates_4bytes)

In [98]:
len(unique_pairs)

2822

In [134]:
len(unique_pairs_4)

1

In [135]:
unique_pairs_4

[(481036462278, 824633837834)]

In [136]:
small_dataset.where('doc_id == 481036462278').take(1)

[Row(TEXT='epithelial OC is a highly fatal disease for which prevention strategies have been limited in part because of our poor understanding of the underlying biology of its precursor the ovarian surface epithelium ose the ose is a single layer of flattocuboidal mesothelial cells that covers the surface of the ovary despite its inconspicuous appearance in vivo it is believed that ose cells actively participate in the cyclical ovulatory rupture and repair process the continuous rupture of the ose at ovulation and the subsequent proliferation to repair the wound renders the cells susceptible to genetic damage and malignant transformation as the ovary is a rich source of multiple hormones and normal ose and ovarian carcinomas secrete and have receptors for hormones growth factors and cytokines these AF are strong candidates to regulate normal ose physiology and the transformation and progression of ovarian cancers in particular alterations of hormonegrowth factor production and receptor

In [137]:
small_dataset.where('doc_id == 824633837834').take(1)

[Row(TEXT='this T0 investigates the antifungal activity of a polymer integrated with nanoporous charcoal particles against paecilomyces variotii chaetomium globosum trichoderma virens which are all filamentous fungi the charcoal polymers were prepared by combining charcoal powders with plastic resin under a vacuum to form charcoal particle protrusions on the polymer surface the mycelial growth of p variotii and t virens exhibited a reduction of and respectively after the conidia were pretreated with charcoal polymers and in particular no mycelial growth was found in c globosum during days of culture the adsorption of ca into charcoal was suggested to inhibit growth due to the reduction in the flux of calcium ions ca into the hyphae in h about mm of ca were removed from cacl solution with gml of polymers and the nanosized pores of the charcoals on the polymer were responsible for the ca adsorption', doc_id=824633837834)]

In [102]:
def pairs(rdd):
    # Function to generate pairs from a list while considering (a, b) the same as (b, a)
    def generate_unique_pairs(values):
        pairs = list(combinations(values, 2))
        #sorted_pairs = [tuple(sorted(pair)) for pair in pairs]
        return pairs

    # Generate pairs for each row
    pairs_rdd = rdd.flatMap(lambda row: generate_unique_pairs(row[1]))

    # Collect all pairs and deduplicate them
    # unique_pairs = pairs_rdd.collect()
    # unique_pairs = list(set(unique_pairs))

    return pairs_rdd

In [103]:
pairs_rdd = pairs(candidates)

In [104]:
pairs_rdd_distinct = pairs_rdd.distinct()

In [105]:
pairs_rdd_distinct.count()

2822

In [107]:
pairs_rdd_distinct.toDF().show()

+------------+------------+
|          _1|          _2|
+------------+------------+
|927713040216|420906861182|
|962072702429|798863979135|
| 77309421895|962072702429|
|481036462278|618475337381|
|824633837834|206158468968|
|738734420393|755914275963|
|755914275963|292057871649|
|558345788776|910533120816|
|575525688293|755914324425|
|712964626235|      106539|
|773094170356|      106539|
|      106539|798864013853|
|558345788776|523986080856|
|214748392950|816043789573|
|712964626235|231928251784|
|712964626235|833223672543|
|635655219536|833223672543|
| 51539642085|738734447526|
|420906861182|893353311394|
|841813690797|712964644856|
+------------+------------+
only showing top 20 rows



In [141]:
small_dataset.where('doc_id == 927713040216').take(1)

[Row(TEXT='leadership research has been encumbered by a proliferation of constructs and measures despite little evidence that each is sufficiently conceptually and operationally distinct from the others we draw from research on subordinates implicit theories of leader SMB behaviorally anchored rating scales and decision making to argue that leader affect ie the degree to which subordinates have positive and negative feelings about their supervisors underlies the common variance shared by many leadership measures to explore this possibility we developed and validated measures of positive and negative leader affect ie the leader affect questionnaires laqs we conducted studies to develop the fiveitem positive and negative laqs and to examine their convergent discriminant predictive and criterionrelated validity we conclude that a the laqs provide highly reliable and valid tools for assessing subordinates evaluations of their leaders b there is significant overlap between existing leadersh

In [139]:
small_dataset.where('doc_id == 420906861182').take(1)['TEXT']

[Row(TEXT='many variables and their interactions can affect a biotechnological process testing a large number of variables and all their possible interactions is a cumbersome task and its cost can be prohibitive several screening strategies with a relatively low number of experiments can be used to find which variables have the largest impact on the process and estimate the magnitude of their effect one approach for process screening is the use of experimental designs among which fractional factorial and plackettburman designs are frequent choices other screening strategies involve the use of artificial neural networks anns the advantage of anns is that they have fewer assumptions than experimental designs but they render blackbox models ie little information can be extracted about the process mechanics in this paper we simulate a biotechnological process fedbatch growth of bakers yeast to analyze and compare the effect of random exp PE of different magnitudes and statistical distribut

In [97]:
for e in unique_pairs:
  print(e)

(712964626235, 25769866227)
(188978671675, 541165895441)
(103079331152, 395136997960)
(523986080856, 292057871649)
(773094170356, 747324402506)
(154618921437, 120259129263)
(652835131901, 816043789573)
(317827674567, 773094170356)
(103079331152, 249108181214)
(747324341244, 472446497290)
(103079293994, 798863979135)
(240518254693, 103079331152)
(730144492409, 712964644856)
(730144492409, 755914275963)
(103079331152, 223338408901)
(824633731178, 798864013853)
(747324341244, 824633731178)
(317827674567, 438086749074)
(498216207321, 927713040216)
(223338408901, 369367283352)
(472446497551, 352187362001)
(455266648102, 369367283352)
(352187362001, 369367283352)
(893353311394, 395137106244)
(120259124985, 154618911990)
(223338408901, 738734447526)
(816043789573, 120259129263)
(927713040216, 833223672543)
(532576006151, 541165929865)
(231928251784, 755914324425)
(738734388598, 833223672543)
(824633837834, 833223672543)
(154618911990, 755914324425)
(798863979135, 755914324425)
(635655219536, 

In [88]:
values = [6,7,8,9]
list(combinations(values, 2))

[(6, 7), (6, 8), (6, 9), (7, 8), (7, 9), (8, 9)]

## NON SO SE MI CONVIENE AVERE UNA LISTA DI PAIRS O RDD CON I PAIRS

In [113]:
df_sig_m = sig_matrix3.toDF()

In [111]:
df_sig_m.show()

+------------+--------------------+
|          _1|                  _2|
+------------+--------------------+
|558345788776|[0, 1, 0, 0, 0, 0...|
|575525688293|[0, 1, 0, 0, 0, 0...|
|240518254693|[0, 1, 0, 0, 0, 0...|
|532576069849|[0, 3, 0, 2, 0, 0...|
|730144492409|[0, 1, 1, 0, 0, 0...|
|962072799544|[0, 1, 3, 2, 0, 2...|
| 77309421895|[0, 1, 0, 0, 0, 0...|
|292057778213|[0, 3, 0, 0, 0, 2...|
|876173412253|[0, 1, 0, 0, 0, 2...|
|712964626235|[0, 1, 0, 0, 0, 0...|
|120259124985|[0, 1, 0, 0, 0, 2...|
|103079248410|[0, 1, 1, 0, 0, 0...|
|498216207321|[0, 3, 0, 0, 0, 0...|
|231928251784|[0, 1, 3, 0, 0, 2...|
|481036462278|[0, 1, 0, 0, 0, 0...|
|618475337381|[0, 1, 0, 1, 0, 2...|
|712964604209|[1, 2, 10, 0, 0, ...|
|214748392950|[0, 1, 0, 0, 0, 0...|
|317827674567|[0, 1, 0, 0, 0, 0...|
|472446497551|[0, 2, 0, 1, 0, 0...|
+------------+--------------------+
only showing top 20 rows



In [122]:
df_sig_m.isLocal()

False

In [114]:
df_sig_m.show()

+------------+--------------------+
|          _1|                  _2|
+------------+--------------------+
|558345788776|[0, 1, 0, 0, 0, 0...|
|575525688293|[0, 1, 0, 0, 0, 0...|
|240518254693|[0, 1, 0, 0, 0, 0...|
|532576069849|[0, 3, 0, 2, 0, 0...|
|730144492409|[0, 1, 1, 0, 0, 0...|
|962072799544|[0, 1, 3, 2, 0, 2...|
| 77309421895|[0, 1, 0, 0, 0, 0...|
|292057778213|[0, 3, 0, 0, 0, 2...|
|876173412253|[0, 1, 0, 0, 0, 2...|
|712964626235|[0, 1, 0, 0, 0, 0...|
|120259124985|[0, 1, 0, 0, 0, 2...|
|103079248410|[0, 1, 1, 0, 0, 0...|
|498216207321|[0, 3, 0, 0, 0, 0...|
|231928251784|[0, 1, 3, 0, 0, 2...|
|481036462278|[0, 1, 0, 0, 0, 0...|
|618475337381|[0, 1, 0, 1, 0, 2...|
|712964604209|[1, 2, 10, 0, 0, ...|
|214748392950|[0, 1, 0, 0, 0, 0...|
|317827674567|[0, 1, 0, 0, 0, 0...|
|472446497551|[0, 2, 0, 1, 0, 0...|
+------------+--------------------+
only showing top 20 rows

